# Lecture29--0410

---

![](img/0410-1.png)

## 隐式序列

built-in range: 需要多少才产生多少

In [1]:
s = range(3, 7)
s

range(3, 7)

In [2]:
len(s)

4

In [7]:
class Range:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def __len__(self):
        return max(0, self.end - self.start)
    
    def __getitem__(self, k):
        if self.start + k >= self.end:
            raise IndexError
        return self.start + k
    
    def __repr__(self):
        return 'Range({0}, {1})'.format(self.start, self.end)
    
s = range(3, 7)
t = Range(3, 7)

In [6]:
for i in t:
    print(i)

3
4
5
6


In [5]:
for i in s:
    print(i)

3
4
5
6


### Interator Interface

+ **An interator is an object that can provide the next element of a sequence.**
+ The **\_\_next\_\_** method of an interator returns the next element.
+ The built-in **next** function invokes the **\_\_next\_\_** method on its argument
+ If there is no next element, then the **\_\_next\_\_** method of an iterator should raise a **StopInteration** exception

In [8]:
si = iter(s)
si.__next__()

3

In [9]:
next(si)

4

In [10]:
next(si)

5

In [11]:
next(si)

6

In [12]:
next(si)

StopIteration: 

In [13]:
class RangeIter:
    def __init__(self, start, end):
        self.next = start
        self.end = end
        
    def __next__(self):
        if self.next >= self.end:
            raise StopIteration
        result = self.next
        self.next += 1
        return result

In [14]:
si = range(3, 7).__iter__()
si

In [15]:
ti = RangeIter(3, 7)
ti

In [22]:
class LetterIter:
    def __init__(self, start, end):
        self.next_letter = start
        self.end = end
        
    def __next__(self):
        if self.next_letter >= self.end:
            raise StopIteration
        result = self.next_letter
        self.next_letter = chr(ord(result)+1)
        return result
    
b_to_e = LetterIter('b', 'e')

In [19]:
next(b_to_e)

'c'

In [20]:
next(b_to_e)

'd'

In [21]:
next(b_to_e)

StopIteration: 

In [24]:
class Letters:
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def __iter__(self):
        return LetterIter(self.start, self.end)
    
li1 = iter(Letters('c', 'd'))
next(li1)

'c'

### Iterators always *lazy computation*

---

![](img/0410-2.png)

~~**\_\_iter\_\_** 函数使得 **map, filter**成立：~~

**map**操作的对象需要实现**\_\_iter\_\_** 函数。

In [25]:
b_to_e = Letters('b', 'e')
m = map(lambda x: x.upper(), b_to_e)
m

In [26]:
next(m)

'B'

In [27]:
list(m)

['C', 'D']

### Lazy

may is lazy

In [32]:
def double(x):
    print('***', x, '=>', 2*x, "***")
    return 2 * x

double(4)

*** 4 => 8 ***


8

In [33]:
m = map(double, range(3, 7))
m

In [34]:
list(m)

*** 3 => 6 ***
*** 4 => 8 ***
*** 5 => 10 ***
*** 6 => 12 ***


[6, 8, 10, 12]

### for 语句

---

![](img/0410-3.png)

---

![](img/0410-4.png)

In [35]:
def letter_generator(next_letter, end):
    while next_letter < end:
        yield next_letter
        next_letter = chr(ord(next_letter)+1)
        
s = letter_generator('a', 'd')
next(s)

'a'

In [36]:
next(s)

'b'

In [37]:
for i in s:
    print(i)

c


# Lecture30--0413

**Warning**: 跳过了使用 *Scheme* 画 *3D* 图像 ———— 前20min

---

### Streams

#### Streams are Lazy Linked Lists

A stream is a linked list, but the rest of the list is computed on demand.

![](img/0413-1.png)

---

# Lecture31--0415 

### http://goo.gl/ajEBkT

### Information Hiding

#### Attributes for Internal Use

An attribute name that starts with on underscore is not meant ro be referenced externally.

**Means**, "Please don't reference these directly. They may change."

In [41]:
class FibIter:
    """An iterator over Fibonacci numbers."""
    def __init__(self):
        self._next = 0
        self._addend = 1
        
    def __next__(self):
        result = self._next
        self._addend, self._next = self._next, self._addend + self._next
        return result
    
fibs = FibIter()
[next(fibs) for _ in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [43]:
def fib_generator():
    """A generator function for Fibonacci numbers."""
    
    yield 0
    previous, current = 0, 1
    while True:
        yield current
        previous, current = current, previous + current
        
fibs = fib_generator()
[next(fibs) for _ in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

### Singleton Objects

In [44]:
class empty_iterator:
    """An iterator over no values."""
    def __next__(self):
        raise StopIteration
        
empty_iterator = empty_iterator()
empty_iterator

In [45]:
next(empty_iterator)

StopIteration: 

---

### Stream Implementation

In [47]:
class Stream:
    """A lazily computed linked list."""
    class empty:
        def __repr__(self):
            return 'Stream.empty'
    empty = empty() 
    
    def __init__(self, first, compute_rest=lambda: Stream.empty):
        assert callable(compute_rest), 'compute_rest must be callable.'
        self.first = first
        self._compute_rest = compute_rest
        
    @property
    def rest(self):
        """Return the rest of the stream, computing it if necessary."""
        if self._compute_rest is not None:
            self._rest = self._compute_rest()
            self._compute_rest = None
        return self._rest

In [48]:
def bad(a, x=1):
    return Stream(x, lambda: bad(a, (x + a/x)/2))

s = bad(2)
s.first

1

In [49]:
s.rest.first

1.5

In [50]:
s.rest._compute_rest

<function __main__.bad.<locals>.<lambda>>

### Declarative Languages

---

![](img/0415-1.png)

### **宣告式编程 v.s. 指令式编程**

![](img/0415-2.png)

---

![](img/0415-3.png)

 # Lecture32--0417
 
 sqlite3 --init ex.sql
 
 ---

![](img/0415-4.png)

# Lecture33--0420

---

![](img/0420-1.png)

---

![](img/0420-2.png)

---

![](img/0420-3.png)

---

![](img/0420-4.png)